Report
Your final report consists of a narrative text that explains what you did, why you did it, and how you went about it. You should break the notebook up into the following meaningful sections so that the instructor can follow the logical flow of your work:

Background and Motivation: Provide some context for the problem and what research question(s) you set out to answer, or the hypothesis you are testing.

Methodology: How did you go about answering your research question(s) or testing your hypothesis? What models did you use and why? How do you know that your model is performing well? Use schematics and figures where necessary to explain concepts.

Results: What did you find when you carried out your methods? Support your work with visuals.

Synthesis and Discussion: What did you learn from your results? What obstacles did you run into? What would you do differently next time? What is the answer to your question(s) and why?

# Predicting the Radius of Gyration of Small Molecules Using Machine Learning

**Author:** Rana Elladki    
**Date:** November 23, 2025

---

## Table of Contents
1. [Background and Motivation](#1-background-and-motivation)
2. [Methodology](#2-methodology)
3. [Results](#3-results)
4. [Synthesis and Discussion](#4-synthesis-and-discussion)

## 1. Background and Motivation

The mean-end-to-end distance is typically sufficient when characterizing the size of linear chains. However, branched and ring polymers have more complex architectures, making the end-to-end distance an insufficient characterization. Therefore, we can use the radius of gyration to describe the size of polymers of any architecture. The radius of gyration squared (RG2) is a structural property of polymers, defined as the average square distances of monomers from the polymer's center of mass. It is mathematically defined as:

$$\left<R_g^2\right> = \frac{1}{N}\sum\limits_{i=1}^{N}\left<\left(\overrightarrow{R}_i-\overrightarrow{R}_{cm}\right)^2\right>$$

where $N$ is the degree of polymerization, $\overrightarrow{R}_i$ is the position vector of molecule $i$, and $\overrightarrow{R}_{cm}$ is the position vector of the center of mass. 

Although RG2 is often associated with polymers, it can also be calculated for small molecules (typically below 300 g/mol). Molecular dynamics and density functional theory can be used to computer RG2, but these methods are computationally expensive, especially for larger systems. Conversely, machine learning provides a way to predict the radius of gyration using physicochemical and topological descriptors instead of performing high-cost simulations. 

Currently, polymer databases lack sufficient data to properly train machine learning (ML) models. Conversely, data for small molecules is readily available. Since polymers and small molcules differ structurally, we can perform a feature-importance analysis to evaluate which physicochemical and topological descriptors have the highest impact on RG2 and whether similar trends apear in polymer RG2 calculations. 

Accurate RG2 predictions can accelerate material design by providing insight into polymer structure, without expensive simulations. Therefore, the main research question this project aims to answer is: "Which supervised machine learning model provides the most accurate predictions of the radius of gyration for small molecules, and which molecular features have the greatest impact on these predictions?"

## 2. Methodology

The small-molecule data was extracted from the QM9 database, which contains computed geometries, energies, and other molecular properties for approximately 134,000 small molecules, each stored in separate `.xyz` files. After data exraction, several pre-processing steps were performed.

First, molecules with a molecular weight greater than 300 g/mol were excluded to restrict this study to small molecules. The atomic coordinates were then extracted from each `.xyz` file to cmpute the RG2. Gibbs free energies were also extracted, to calculate the Boltzmann weights.

The Boltzmann weights were initially calculated to group conformers together. However, the QM9 dataset did not contain any conformers. Therefore, the RG2 calculations for each molecule depend only on its report geometry. After confirming the absence of conformers, we constructed a dictionary of descriptors (features) and targets (RG2).

Physicochemical descriptors were computed using RDKit, ands serveral additional topological descriptors were manually calculated. 

The Wiener index(WI) is defined as:

$$WI = \sum\limits_{i\neq j}d(i,j)$$

where $d(i,j)$ is the shortest path between molecules $i$ and $j$. WI reflects the global shape and size of the molecule. 

Tthe Randic index (RI) was calculated as:

$$RI = \sum\limits_{i,j\in E} \frac{1}{\sqrt{d_id_j}}$$

where $d_i$, $d_j$ are the degrees of vertices $i$ and $j$ (number of neighbors), and $E$ is the set of edges (bonds). RI measures us on local connectivity and is inversely related to branching. 

The Zagreb indices (Z1 and Z2) were also computed:

$$Z1 = \sum\limits_i d_i^2$$

$$Z2 = \sum\limits_{i,j\in E} d_id_j$$

Z1 and Z2 characterize degree distribution and used as indicators of molecular branching. 

Lastly, we calculated the radius of gyration, using the mathematical formula shown in the background. 

After building the descriptor dictionary, we performed exploratory data analysis (EDA). We first computed the Pearson correlation coefficient (PCC) between each descriptor and RG2 to identify strong relationships. A PCC of positive one indicates a perfect positive correlation, a negative one indicates a perfect negative correlation, and zero indicated no correlation. We retained the top 20 descriptors with highest absolute PCC values. 

Outliers can distort model training. Since we want to understand the relationship of the features with the target property, we filter out outliers, possibly skewing that relationship.

This concludes our exploratory data analysis, so we split the data into training, validation, and testing sets.
First, we split the data randomly into 80% for training and 20% for testing. Then, the training set was split randomly again, with 75% for training and 25% for validation. The final split for the training, validation, and testing sets, was 60%, 20%, and 20% of the filtered data respectively. Five total machine learning models were trainined. 

The first set of models were linear regression models, implemented using LinearRegression from the scikit learn linear_model library. The first model was trained on the filtered data without any additional transformation. A second linear model was trained after applying a logarithmic transformation to skewed features.

Three additional models were artificial neural networks (ANNs), implemented using MLPRegressor from the scikit learn neural network, which implements a feed-forward artificial neural network. We also used GridSearchCV to perform hyperparameter tuning using 5-fold cross validation, to find the best model configuration. The three ANN variations were trained on:

1. filtered data only,
2. log-tranformed feature data,
3. standardized feature data using StandardScaler.

After training, we evaluated all models using standard statistical metrics:

1. Coefficient of Determination ($R^2$):
    Measures how well the data fits the model
2. Mean Squared Error (MSE):
    Measures the average of the squared differences between predicted and true values
3. Root Mean Squared Error (RMSE):
    Square root of MSE. Assigns a smaller penalty on deviations compared to MSE.
4. Mean Absolute Error (MAE):
    Measures the differences between predicted and true values. Less sensitive to outliers.
5. Pearson Correlation (PC):
    Evaluate the correlation between the predicted and true values.    

## 3. Results  

We first verified that all molecules in the QM9-derived dataset satisfied the molecular weight cutoff of 300 g/mol. As shown in the histogram below, the entire distribution lies well below this threshold, so no molecules were excluded.

![wts_dist.png](./wts_dist.png)

After extracting physicochemical descriptors from RDKit and computing additional topological descriptors, our dataset contained over 200 features. Therefore, we performed a Pearson correlation coefficient (PCC) analysis to select the top 20 features most correlated with RG2, to reduce the possibility of noise. The heatmap below shows the feature-target relationships and the inter-feature correlations.

![corr_heatmap_top_20_feats.png](./corr_heatmap_top_20_feats.png)

The heatmap also shows the relationship of the features with each other. For example, Phi and Kappa2 have a 0.99 PCC, indicating a strong positive relationship. Most importantly, all selected descriptors exhibited a non-zero PCC with RG2, confirming that each feature has an impact on our target and should be captured in our traind models. 

TO examine feature distributions, we plotted the histograms of our raw feature and target data:

![hist_unfiltered.png](./hist_unfiltered.png)

Outliers can result in heavily skewed distributions for some discriptors, so we removed values below the 0.1th percentile and above the 99.9th percentile. After filtering, several features, such as Kappa2, Phi, and Chi0, show visibly shorter tails and reduced skew:

![hist_filtered.png](./hist_filtered.png)

Although filtering reduced skewness, some features remained far from Gaussian. Therefore, we applied a logarithmic transform to nonzero features within the training, validation, and test splits. Features containing true zeros, such as NumRotatableBons and NumSaturatedRings, were left untransformed to avoid discarding valid molecules. Below are the histograms for the logged features from the training set:

![hist_logged.png](./hist_logged.png)

Upon completing the exploratory data analysis and dataset splitting, we trained two linear regression (LR) models and three artificial neural networks (ANNs). For ease of referencing, we will refer to the models as follows:

* Model 1: LR model trained on filtered dataset
* Model 2: LR model trained on logged dataset
* Model 3: ANN model trained on filtered dataset
* Model 4: ANN model trained on logged dataset
* Model 5: ANN model trained on standardized features dataset (StandardScaler)

The following hyperparameters were optimized using GridSearchCV with 5-fold cross-validation for our ANNs:

* `hidden_layer_size`: (10,), (10, 20), (20,30), (10, 20, 30)
* `batch_size`: 100, 200, 500
* `learning_rate_init`: 0.01, 0.05, 0.

All five models were evaluated using $R^2$, RMSE, MSE, MAE, AND PC on the validation and testing set, as summarized below:

| Model                           | Scaling |   R²  |  RMSE |  MSE  |  MAE  |   PC  |
|---------------------------------|---------|-------|-------|-------|-------|-------|
| **--- VALIDATION SET ---**      |         |       |       |       |       |       |
| Linear Regression (LR)          |   --    | 0.819 | 0.434 | 0.189 | 0.312 | 0.905 |
| Linear Regression (LR)          |   Log   | 0.844 | 0.403 | 0.162 | 0.283 | 0.919 |
| Artificial Neural Network (ANN) |   --    | 0.789 | 0.469 | 0.220 | 0.329 | 0.906 |
| Artificial Neural Network (ANN) |   Log   | 0.890 | 0.338 | 0.114 | 0.233 | 0.949 |
| Artificial Neural Network (ANN) | Scaled  | 0.927 | 0.275 | 0.076 | 0.186 | 0.963 |
| **--- TESTING SET ---**         |         |       |       |       |       |       |
| Linear Regression (LR)          |   --    | 0.819 | 0.432 | 0.186 | 0.314 | 0.905 |
| Linear Regression (LR)          |   Log   | 0.844 | 0.400 | 0.160 | 0.285 | 0.919 |
| Artificial Neural Network (ANN) |   --    | 0.789 | 0.466 | 0.217 | 0.330 | 0.904 |
| Artificial Neural Network (ANN) |   Log   | 0.888 | 0.340 | 0.116 | 0.235 | 0.948 |
| Artificial Neural Network (ANN) | Scaled  | 0.926 | 0.277 | 0.077 | 0.189 | 0.963 |


The ANN models outperformed LR models across all metrics. The model trained on standardized features achieved the best performance overall, with an $R^2$ of 0.926 on the testing set and the lowest error values.

Next, we can take a look at the predicted versus true plots for the two LR models:

![lr_pred_vs_true.png](./lr_pred_vs_true.png)

Both LR models struggled to predict large RG2 values (>8), likely due to the dataset containing relatively few molecules with high RG2 values. The log-transformed LR model perfomed better but still showed underpredictions at the upper range.

The ANN predictions show a similar trend, with Model 3 underpredicting large RG2 values, but with significantly reduced deviation in Models 4 and 5:

![ann_pred_vs_true.png](./ann_pred_vs_true.png)

Model 3 exhibits the largest spread, while Models 4 and 5 align much more closely with the identity line. Model 5 provides the tightest clustering and smallest deviations, indicating superior accuracy in predicting realistic RG2 values. 

We also examined the coefficient magnitudes for the best LR model and conducted a SHAP analysis for the best ANN model, to compare feature indluence on RG2 predictions.

The coefficient weights for the 20 features used in Model 3:

![lr_coef_wghts.png](./lr_coef_wghts.png)

The SHAP scores for the 20 features used in Model 5:

![ann_shap.png](./ann_shap.png)

Comparing the LR coefficent weights to the ANN SHAP scores, both models identified Kappa2 as the strongest feature negatively correlated with RG2, which is consistent with its interpretation as a measure of molecular branching compactness. However, several features showed model-dependent importance. The ANN assigned high importance to WI and Kappa1, while LR assigned them a relatively low weight. This suggests that the ANN was able to capture nonliner relationships between topological descriptors and RG2 that the LR could not. 

From a polymer-physics perspective, WI should correlate strongly with RG2, since both increase with molecular branching. Since the ANN relfects this physical trend more clearly than LR, indicates that the ANN is not only statistically superior but also more scientifically meaningful. 

## 4. Synthesis and Discussion

The goal of this project was to determine which supervised machine learning model provides the most accurate predictions of the radius of gyration for small molcules, and to identify the molecular descriptors that most strongly influence RG2. We trained two linear regression models, one on the filtered dataset and one of the log-transformed features, and three artificial neural networks:
1. ANN with filtered features
2. ANN with log-transformed features
3. ANN with standardized features

The results show that models ANN models incorporating feature scaling (logged/standardized) significantly outperformed linear models. 

A key finding is that the most influential features, Kappa2, WI, Kappa1, Chi0, and Z1, are all topological descriptors. These quantities reflect molecular connectivity, branching, and graph distances, rather than chemical composition. This suggests that the RG2 for small molecules is primarily governed by molecular shape and size, which is consistent with polymer physics where RG2 is strongly dependent on global architecture. The ANN models trained on scaled and logged features captures these relationships more effectively than linear regression, explaining their super performance. 

Model 5, the ANN trained on standardized features, achieved the highest accuracy with an $R^2$ of 0.93 on the test set and the lowest error values overall. The optimal architecture for this model is two hidden layers with sizes 20 and 30 for the first and second layer, respectively, a batch_size of 500, and a learning rate of 0.01. 

Several challenges were encountered throughout this project. First, many descriptors exhibited highly skewed distributions, requiring filtering and transformations, to minimize possible bias. Second, the dataset contains few molecules with large RG2 values, which possibly contributed to the underpridiction in both LR models and the ANN model trained on filetered features, with no additional scaling. Lastly, hyperparamter tuning using GridSearchCV was computationally expensive, especially when increasing the number and size of hidden layers for our ANNs.

In future work, several improvements could be implemented to strengthen model performance. First, using models trained on molecular graphs, such as graph neural networks (GNNs) and convolution neural networks (CNNs), may further enhance accuracy, since these approaches learn from topological descriptors rather than chemical descriptors. Additionally, expanding the dataset to include conformers and incoporating Botlzmann-weighted RG2 averages would extend this beyond the QM9 database, which did not include any conformers. Finally, a dataset with a broader RG2 coverage would help improve predictions in the high-RG2 regime.

Overall, the ANN model trained on standardized features provided the predictions, both statistically and physically, demonstating that the radius of gyration is most strongly influenced by topological descriptors. Expanding our database to larger molecules and using models trained on topological descriptors can possibly allow us to extend our model beyond small molecules and into the polymer region. 